# 🧹 Nettoyage des Données - Hackathon Babyfoot

Ce notebook réalise le nettoyage complet du dataset `babyfoot_dataset.csv`.

## Objectifs
- Nettoyer les dates, scores, noms, âges, rôles, couleurs
- Supprimer les doublons
- Gérer les valeurs manquantes
- Produire un dataset propre et exploitable

**Auteurs**: HAMMADI Otmane & EL ARJOUNI Mohamed Amine  
**Date**: 16/10/2025

## 📦 Imports et Configuration

In [23]:
import pandas as pd
import numpy as np
from dateutil import parser
import re
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports réussis!")

✅ Imports réussis!


## 1️⃣ Chargement des Données

In [24]:
# Charger le dataset brut
df = pd.read_csv('../../ressources/babyfoot_dataset.csv')

print(f"📊 Dataset chargé avec succès!")
print(f"   Nombre de lignes: {len(df)}")
print(f"   Nombre de colonnes: {len(df.columns)}")
print(f"   Parties uniques: {df['game_id'].nunique()}")
print(f"   Joueurs uniques: {df['player_id'].nunique()}")

# Aperçu
df.head()

📊 Dataset chargé avec succès!
   Nombre de lignes: 100200
   Nombre de colonnes: 35
   Parties uniques: 25002
   Joueurs uniques: 803


,game_id,game_date,location,table_id,table_condition,ball_type,music_playing,referee,game_duration,final_score_red,...,possession_time,mood,player_comment,team_color,is_substitute,ping_ms,notes,duplicate_flag,misc,created_at
0,G015295,Feb 06st 2023,Ynov Toulouse,T05,beer stains,NaN,Spotify: Queen - We Will Rock You,Paul Kim,12.45,0,...,9:36,1,ref biased,Red,yes,185,injured,NaN,NaN,2025-10-02T10:41:54
1,G023800,24-03-2023,Cafeteria (1st floor),T07,worn,NaN,Indie playlist,NaN,8.57,10,...,NaN,🙂,NaN,Blue,yes,NaN,NaN,0,-,2025-10-02T10:41:55
2,G023577,2025-01-13,Gym Hall,T26,scratched,NaN,Spotify: Queen - We Will Rock You,Lena Clement,17.2,2,...,5.09min,2,NaN,R,no,NaN,double booked,0,NaN,2025-10-02T10:41:55
3,G020644,Nov 11 2025,Salle Polyvalente,T21,worn,mini ball,EDM mix,Isabella Girard,5.18,6,...,NaN,3,team spirit high,B,maybe,NaN,double booked,NaN,-,2025-10-02T10:41:54
4,G011677,30 Sep 23,Campus - Cafeteria,T26,missing screw,trainer ball,Oldies 80s,yes,6min,3,...,177,😂,rage quit,Red,yes,NaN,NaN,NaN,NaN,2025-10-02T10:41:54


## 2️⃣ Analyse de la Qualité des Données

Identifions les problèmes avant de nettoyer.

In [25]:
# Valeurs manquantes
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Colonne': missing.index,
    'Valeurs Manquantes': missing.values,
    'Pourcentage': missing_pct.values
})

print("🔍 Colonnes avec valeurs manquantes:\n")
missing_df[missing_df['Valeurs Manquantes'] > 0].sort_values('Pourcentage', ascending=False)

🔍 Colonnes avec valeurs manquantes:



,Colonne,Valeurs Manquantes,Pourcentage
33,misc,60128,60.01
7,referee,60083,59.96
30,ping_ms,50176,50.08
19,player_age,39887,39.81
32,duplicate_flag,39867,39.79
31,notes,33387,33.32
29,is_substitute,25176,25.13
25,possession_time,25052,25.00
5,ball_type,22525,22.48
27,player_comment,16587,16.55


In [26]:
# Exemples de problèmes de qualité
print("🔍 Problèmes identifiés:\n")

print("1. Formats de dates variés:")
print(df['game_date'].head(5).tolist())

print("\n2. Formats de scores variés:")
print(df[['final_score_red', 'final_score_blue']].head(5).to_string())

print("\n3. Noms avec caractères spéciaux:")
special_names = df[df['player_name'].str.contains('3|0', na=False, regex=True)]['player_name'].head(5)
print(special_names.tolist())

print("\n4. Variantes de winner:")
print(df['winner'].value_counts().head(10))

🔍 Problèmes identifiés:

1. Formats de dates variés:
['Feb 06st 2023', '24-03-2023', '2025-01-13', 'Nov 11 2025', '30 Sep 23']

2. Formats de scores variés:
  final_score_red final_score_blue
0               0              4.0
1              10              7.0
2               2              6.0
3               6              9.0
4               3             10.0

3. Noms avec caractères spéciaux:
['Juli3 Mor3au', 'Cas3y Kim', 'Paul Cl3m3nt', 'Al3x Mor3l', 'Ethan M3rci3r']

4. Variantes de winner:
winner
Red      9974
Blue     9798
RED      8961
red      8799
B        8747
Bleu     8704
Rouge    8683
R        8610
BLUE     8528
blue     8404
Name: count, dtype: int64
['Juli3 Mor3au', 'Cas3y Kim', 'Paul Cl3m3nt', 'Al3x Mor3l', 'Ethan M3rci3r']

4. Variantes de winner:
winner
Red      9974
Blue     9798
RED      8961
red      8799
B        8747
Bleu     8704
Rouge    8683
R        8610
BLUE     8528
blue     8404
Name: count, dtype: int64


## 3️⃣ Nettoyage des Dates

Convertir tous les formats de dates vers un format standard.

In [27]:
def parse_date(date_str):
    """Parse une date avec gestion des formats variés"""
    if pd.isna(date_str):
        return None
    
    try:
        # Nettoyer les formats bizarres (Feb 06st, Mar 25st, etc.)
        date_str = str(date_str)
        date_str = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)
        
        # Parser la date
        dt = parser.parse(date_str, dayfirst=False)
        return dt
    except:
        return None

# Appliquer le nettoyage
dates_before = df['game_date'].isna().sum()
df['game_date_cleaned'] = df['game_date'].apply(parse_date)
dates_after = df['game_date_cleaned'].isna().sum()

print(f"📅 Nettoyage des dates:")
print(f"   Dates invalides avant: {dates_before}")
print(f"   Dates invalides après: {dates_after}")
print(f"   ✅ {dates_before - dates_after} dates converties")

# Aperçu
df[['game_date', 'game_date_cleaned']].head(10)

📅 Nettoyage des dates:
   Dates invalides avant: 0
   Dates invalides après: 1
   ✅ -1 dates converties


,game_date,game_date_cleaned
0,Feb 06st 2023,2023-02-06
1,24-03-2023,2023-03-24
2,2025-01-13,2025-01-13
3,Nov 11 2025,2025-11-11
4,30 Sep 23,2023-09-30
5,2023/07/04,2023-07-04
6,2023/11/16,2023-11-16
7,2025/11/16,2025-11-16
8,20 Mar 23,2023-03-20
9,04 Feb 24,2024-02-04


## 4️⃣ Nettoyage des Scores

Extraire les scores numériques propres.

In [28]:
def extract_score(score_str):
    """Extrait le score numérique d'une chaîne"""
    if pd.isna(score_str):
        return None
    
    score_str = str(score_str).strip()
    
    # Cas "5 - 3" ou "5-3"
    if '-' in score_str:
        parts = score_str.split('-')
        try:
            return int(parts[0].strip())
        except:
            return None
    
    # Cas simple "5"
    try:
        return int(score_str)
    except:
        return None

# Appliquer le nettoyage
scores_before = df['final_score_red'].isna().sum() + df['final_score_blue'].isna().sum()

df['final_score_red_cleaned'] = df['final_score_red'].apply(extract_score)
df['final_score_blue_cleaned'] = df['final_score_blue'].apply(extract_score)

scores_after = df['final_score_red_cleaned'].isna().sum() + df['final_score_blue_cleaned'].isna().sum()

print(f"🎯 Nettoyage des scores:")
print(f"   Scores invalides avant: {scores_before}")
print(f"   Scores invalides après: {scores_after}")
print(f"   ✅ {scores_before - scores_after} scores nettoyés")

# Aperçu
df[['final_score_red', 'final_score_red_cleaned', 'final_score_blue', 'final_score_blue_cleaned']].head(10)

🎯 Nettoyage des scores:
   Scores invalides avant: 4906
   Scores invalides après: 84563
   ✅ -79657 scores nettoyés


,final_score_red,final_score_red_cleaned,final_score_blue,final_score_blue_cleaned
0,0,0.0,4.0,NaN
1,10,10.0,7.0,NaN
2,2,2.0,6.0,NaN
3,6,6.0,9.0,NaN
4,3,3.0,10.0,NaN
5,9,9.0,8.0,NaN
6,0,0.0,4.0,NaN
7,7,7.0,7.0,NaN
8,8,8.0,7.0,NaN
9,0,0.0,5.0,NaN


## 5️⃣ Standardisation du Gagnant

In [29]:
def standardize_winner(winner_str):
    """Standardise le nom du gagnant"""
    if pd.isna(winner_str):
        return None
    
    winner_str = str(winner_str).strip().lower()
    
    # Équipe Rouge
    if winner_str in ['red', 'rouge', 'r', 'red team']:
        return 'Red'
    
    # Équipe Bleue
    if winner_str in ['blue', 'bleu', 'b', 'blue team']:
        return 'Blue'
    
    # Match nul
    if winner_str in ['draw', 'tie', 'égalité', '']:
        return 'Draw'
    
    return None

# Appliquer
winners_before = df['winner'].isna().sum()
df['winner_cleaned'] = df['winner'].apply(standardize_winner)
winners_after = df['winner_cleaned'].isna().sum()

print(f"🏆 Standardisation des gagnants:")
print(f"   Avant: {df['winner'].value_counts().head(10).to_dict()}")
print(f"\n   ✅ Après standardisation:")
print(f"   {df['winner_cleaned'].value_counts().to_dict()}")

# Aperçu
df[['winner', 'winner_cleaned']].head(10)

🏆 Standardisation des gagnants:
   Avant: {'Red': 9974, 'Blue': 9798, 'RED': 8961, 'red': 8799, 'B': 8747, 'Bleu': 8704, 'Rouge': 8683, 'R': 8610, 'BLUE': 8528, 'blue': 8404}

   ✅ Après standardisation:
   {'Red': 45027, 'Blue': 44181, 'Draw': 6321}


,winner,winner_cleaned
0,Blue,Blue
1,red,Red
2,Bleu,Blue
3,Blue,Blue
4,blue,Blue
5,Red,Red
6,blue,Blue
7,NaN,None
8,R,Red
9,B,Blue


## 6️⃣ Nettoyage des Noms de Joueurs

Utiliser le nom canonique et corriger les caractères spéciaux.

In [30]:
def clean_name(name_str):
    """Nettoie un nom de joueur"""
    if pd.isna(name_str):
        return None
    
    name_str = str(name_str)
    # Remplacer les caractères spéciaux (3 -> e, etc.)
    name_str = name_str.replace('3', 'e')
    name_str = name_str.replace('0', 'o')
    name_str = name_str.replace('1', 'i')
    
    # Capitaliser correctement
    name_str = name_str.title()
    
    return name_str

# Utiliser player_canonical_name si disponible, sinon nettoyer player_name
df['player_name_cleaned'] = df.apply(
    lambda row: row['player_canonical_name'] if pd.notna(row['player_canonical_name']) 
    else clean_name(row['player_name']),
    axis=1
)

print(f"👤 Nettoyage des noms:")
print(f"   ✅ {len(df)} noms traités")
print(f"\n   Exemples de noms nettoyés:")

# Comparer avant/après
comparison = df[df['player_name'] != df['player_name_cleaned']][['player_name', 'player_name_cleaned']].head(10)
comparison

👤 Nettoyage des noms:
   ✅ 100200 noms traités

   Exemples de noms nettoyés:


,player_name,player_name_cleaned
0,Juli3 Mor3au,Julie Moreau
1,mohamed philippe,Mohamed Philippe
2,luc rossi,Luc Rossi
3,Morgan R.,Morgan Rossi
4,Cas3y Kim,Casey Kim
5,C. Blanc,Chris Blanc
6,Garcia Antoine,Antoine Garcia
7,Paul Cl3m3nt,Paul Clement
8,Pierre,Pierre Durand
9,Rousseau Morgan,Morgan Rousseau


## 7️⃣ Conversion des Âges

In [31]:
def parse_age(age_str):
    """Parse un âge depuis différents formats"""
    if pd.isna(age_str):
        return None
    
    age_str = str(age_str).lower().strip()
    
    # Mots en anglais
    age_words = {
        'twenty': 20, 'twenty-one': 21, 'twenty-two': 22,
        'sixteen': 16, 'seventeen': 17, 'eighteen': 18, 'nineteen': 19
    }
    
    if age_str in age_words:
        return age_words[age_str]
    
    # Extraire le nombre (ex: "21 yrs", "21")
    match = re.search(r'\d+', age_str)
    if match:
        age = int(match.group())
        # Validation basique (âge entre 15 et 100)
        if 15 <= age <= 100:
            return age
    
    return None

# Appliquer
ages_before = df['player_age'].isna().sum()
df['player_age_cleaned'] = df['player_age'].apply(parse_age)
ages_after = df['player_age_cleaned'].isna().sum()

print(f"🎂 Conversion des âges:")
print(f"   Âges invalides avant: {ages_before}")
print(f"   Âges invalides après: {ages_after}")
print(f"   ✅ {ages_before - ages_after} âges convertis")
print(f"\n   Statistiques des âges:")
print(f"   Moyenne: {df['player_age_cleaned'].mean():.1f} ans")
print(f"   Min: {df['player_age_cleaned'].min()}, Max: {df['player_age_cleaned'].max()}")

# Aperçu
df[['player_age', 'player_age_cleaned']].head(10)

🎂 Conversion des âges:
   Âges invalides avant: 39887
   Âges invalides après: 39889
   ✅ -2 âges convertis

   Statistiques des âges:
   Moyenne: 23.8 ans
   Min: 16.0, Max: 45.0


,player_age,player_age_cleaned
0,twenty,20.0
1,NaN,NaN
2,21 yrs,21.0
3,21 yrs,21.0
4,26,26.0
5,21 yrs,21.0
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,34,34.0


## 8️⃣ Standardisation des Rôles et Couleurs

In [32]:
def standardize_role(role_str):
    """Standardise le rôle du joueur"""
    if pd.isna(role_str):
        return None
    
    role_str = str(role_str).lower().strip()
    
    # Attaquant
    if role_str in ['attack', 'attck', 'attaque', 'forward']:
        return 'Attack'
    
    # Défenseur
    if role_str in ['defence', 'defense', 'def', 'défense']:
        return 'Defense'
    
    return None

def standardize_team_color(color_str):
    """Standardise la couleur de l'équipe"""
    if pd.isna(color_str):
        return None
    
    color_str = str(color_str).lower().strip()
    
    # Rouge
    if color_str in ['red', 'rouge', 'r', '🔴']:
        return 'Red'
    
    # Bleu
    if color_str in ['blue', 'bleu', 'b', '🔵']:
        return 'Blue'
    
    return None

# Appliquer
df['player_role_cleaned'] = df['player_role'].apply(standardize_role)
df['team_color_cleaned'] = df['team_color'].apply(standardize_team_color)

print(f"⚽ Standardisation des rôles:")
print(f"   {df['player_role_cleaned'].value_counts().to_dict()}")

print(f"\n🎨 Standardisation des couleurs:")
print(f"   {df['team_color_cleaned'].value_counts().to_dict()}")

⚽ Standardisation des rôles:
   {'Defense': 57358, 'Attack': 42841}

🎨 Standardisation des couleurs:
   {'Red': 50119, 'Blue': 50077}


## 9️⃣ Conversion des Durées en Secondes

In [33]:
def parse_duration(duration_str):
    """Parse une durée et la convertit en secondes"""
    if pd.isna(duration_str):
        return None
    
    duration_str = str(duration_str).strip().lower()
    
    try:
        # Format "00:10:23" (HH:MM:SS)
        if duration_str.count(':') == 2:
            parts = duration_str.split(':')
            hours = int(parts[0])
            minutes = int(parts[1])
            seconds = int(parts[2])
            return hours * 3600 + minutes * 60 + seconds
        
        # Format "10:23" (MM:SS)
        elif duration_str.count(':') == 1:
            parts = duration_str.split(':')
            minutes = int(parts[0])
            seconds = int(parts[1])
            return minutes * 60 + seconds
        
        # Format "12min"
        elif 'min' in duration_str:
            minutes = float(duration_str.replace('min', ''))
            return int(minutes * 60)
        
        # Format décimal (minutes)
        else:
            minutes = float(duration_str)
            return int(minutes * 60)
    except:
        return None

# Appliquer
durations_before = df['game_duration'].isna().sum()
df['game_duration_seconds'] = df['game_duration'].apply(parse_duration)
durations_after = df['game_duration_seconds'].isna().sum()

print(f"⏱️  Conversion des durées:")
print(f"   Durées invalides avant: {durations_before}")
print(f"   Durées invalides après: {durations_after}")
print(f"   ✅ {durations_before - durations_after} durées converties")
print(f"\n   Durée moyenne: {df['game_duration_seconds'].mean()/60:.1f} minutes")

# Aperçu
df[['game_duration', 'game_duration_seconds']].head(10)

⏱️  Conversion des durées:
   Durées invalides avant: 0
   Durées invalides après: 1
   ✅ -1 durées converties

   Durée moyenne: 17.5 minutes


,game_duration,game_duration_seconds
0,12.45,747.0
1,8.57,514.0
2,17.2,1032.0
3,5.18,310.0
4,6min,360.0
5,5min,300.0
6,18min,1080.0
7,00:08:11,491.0
8,14.62,877.0
9,16.11,966.0


## 🔟 Standardisation des Saisons

Convertir tous les formats de saisons vers un format standardisé YYYY-YYYY.

In [34]:
def standardize_season(season_value):
    """
    Convertit différents formats de saison en format standardisé YYYY-YYYY

    Exemples:
        '2023/2024' -> '2023-2024'
        'Season 24-25' -> '2024-2025'
        '2025 Season' -> '2025-2026'
        's24/25' -> '2024-2025'
        '23/24' -> '2023-2024'
    """
    if pd.isna(season_value):
        return None

    season_str = str(season_value).strip()

    # Format: 2023/2024, 2023-2024 (années complètes)
    match = re.search(r'(20\d{2})[\/-](20\d{2})', season_str)
    if match:
        return f"{match.group(1)}-{match.group(2)}"

    # Formats compacts avec années sur 2 chiffres et préfixe optionnel 'Season' ou 's'
    # Ex: 'Season 24-25', 's24/25', '24-25', '24/25'
    match = re.search(r'^(?:[sS](?:eason)?\s*)?(\d{2})[\/-](\d{2})$', season_str)
    if match:
        year1 = match.group(1)
        year2 = match.group(2)
        full_year1 = f"20{year1}"
        full_year2 = f"20{year2}"
        return f"{full_year1}-{full_year2}"

    # Format: 2025 Season (implique 2025-2026)
    match = re.search(r'(20\d{2})\s*[Ss]eason', season_str)
    if match:
        year1 = int(match.group(1))
        year2 = year1 + 1
        return f"{year1}-{year2}"

    # Si seule une année: 2025 -> 2025-2026
    match = re.search(r'^(20\d{2})$', season_str)
    if match:
        year1 = int(match.group(1))
        year2 = year1 + 1
        return f"{year1}-{year2}"

    # Si rien ne correspond, retourner la valeur originale
    return season_str

# Afficher les formats de saisons avant nettoyage
print(f"📅 Standardisation des saisons:")
print(f"   Valeurs uniques avant: {df['season'].nunique()}")
print(f"   Exemples avant nettoyage:")
print(f"   {df['season'].unique()[:10].tolist()}")

# Appliquer la standardisation
seasons_before = df['season'].isna().sum()
df['season_cleaned'] = df['season'].apply(standardize_season)
seasons_after = df['season_cleaned'].isna().sum()

print(f"\n   ✅ Valeurs uniques après: {df['season_cleaned'].nunique()}")
print(f"   Exemples après nettoyage:")
print(f"   {df['season_cleaned'].unique()[:10].tolist()}")

print(f"\n   Saisons invalides avant: {seasons_before}")
print(f"   Saisons invalides après: {seasons_after}")

# Aperçu
df[['season', 'season_cleaned']].head(10)

📅 Standardisation des saisons:
   Valeurs uniques avant: 5
   Exemples avant nettoyage:
   ['2023/2024', 'Season 24-25', '2024/2025', '2025 Season', 's24/25']

   ✅ Valeurs uniques après: 3
   Exemples après nettoyage:
   ['2023-2024', '2024-2025', '2025-2026']

   Saisons invalides avant: 0
   Saisons invalides après: 0

   ✅ Valeurs uniques après: 3
   Exemples après nettoyage:
   ['2023-2024', '2024-2025', '2025-2026']

   Saisons invalides avant: 0
   Saisons invalides après: 0


,season,season_cleaned
0,2023/2024,2023-2024
1,Season 24-25,2024-2025
2,2024/2025,2024-2025
3,2024/2025,2024-2025
4,2025 Season,2025-2026
5,2023/2024,2023-2024
6,2024/2025,2024-2025
7,s24/25,2024-2025
8,2025 Season,2025-2026
9,2023/2024,2023-2024


## 🔟 Suppression des Doublons

In [35]:
# Suppression des doublons basée sur game_id et player_id
before = len(df)
df = df.drop_duplicates(subset=['game_id', 'player_id'], keep='first')
after = len(df)
duplicates_removed = before - after

print(f"🔄 Suppression des doublons:")
print(f"   Lignes avant: {before}")
print(f"   Lignes après: {after}")
print(f"   ✅ {duplicates_removed} doublons supprimés")

🔄 Suppression des doublons:
   Lignes avant: 100200
   Lignes après: 100000
   ✅ 200 doublons supprimés


## 1️⃣1️⃣ Traitement des Valeurs Manquantes et Création du Dataset Final

In [36]:
# Pour les statistiques numériques, remplacer par 0 si c'est logique
numeric_stats = ['player_goals', 'player_own_goals', 'player_assists', 'player_saves']
for col in numeric_stats:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# Marquer les lignes avec trop de données manquantes
critical_cols = ['game_id', 'player_id', 'player_name_cleaned']
df['is_valid'] = df[critical_cols].notna().all(axis=1)

invalid_count = (~df['is_valid']).sum()
print(f"❓ Traitement des valeurs manquantes:")
print(f"   Statistiques numériques: remplies avec 0")
print(f"   ⚠️  {invalid_count} lignes marquées comme invalides (données critiques manquantes)")

# Créer le dataset nettoyé final - utiliser les colonnes qui existent réellement
cleaned_columns = {
    'game_id': 'game_id',
    'game_date_cleaned': 'game_date',
    'location': 'location',
    'table_id': 'table_id',
    'game_duration_seconds': 'game_duration_seconds',
    'final_score_red_cleaned': 'final_score_red',
    'final_score_blue_cleaned': 'final_score_blue',
    'winner_cleaned': 'winner',
    'season_cleaned': 'season',  # Utiliser season_cleaned
    'player_id': 'player_id',
    'player_name_cleaned': 'player_name',
    'player_age_cleaned': 'player_age',
    'player_role_cleaned': 'player_role',
    'player_goals': 'player_goals',
    'player_own_goals': 'player_own_goals',
    'player_assists': 'player_assists',
    'player_saves': 'player_saves',
    'team_color_cleaned': 'team_color',
    'is_valid': 'is_valid'
}

# Vérifier quelles colonnes existent réellement
existing_cols = {k: v for k, v in cleaned_columns.items() if k in df.columns}
print(f"\n📋 Colonnes disponibles: {len(existing_cols)}/{len(cleaned_columns)}")

# Créer le dataframe avec les colonnes existantes
df_cleaned = df[list(existing_cols.keys())].rename(columns=existing_cols)

# Garder seulement les lignes valides
df_final = df_cleaned[df_cleaned['is_valid'] == True].drop(columns=['is_valid'])

print(f"\n✅ Dataset final créé:")
print(f"   Lignes valides: {len(df_final)}")
print(f"   Colonnes: {len(df_final.columns)}")

❓ Traitement des valeurs manquantes:
   Statistiques numériques: remplies avec 0
   ⚠️  0 lignes marquées comme invalides (données critiques manquantes)

📋 Colonnes disponibles: 19/19

✅ Dataset final créé:
   Lignes valides: 100000
   Colonnes: 18

✅ Dataset final créé:
   Lignes valides: 100000
   Colonnes: 18


## 📊 Aperçu du Dataset Nettoyé

In [37]:
# Aperçu des premières lignes
print("📋 Premières lignes du dataset nettoyé:\n")
df_final.head(10)

📋 Premières lignes du dataset nettoyé:



,game_id,game_date,location,table_id,game_duration_seconds,final_score_red,final_score_blue,winner,season,player_id,player_name,player_age,player_role,player_goals,player_own_goals,player_assists,player_saves,team_color
0,G015295,2023-02-06,Ynov Toulouse,T05,747.0,0.0,NaN,Blue,2023-2024,P0382,Julie Moreau,20.0,Defense,0,0,4,3,Red
1,G023800,2023-03-24,Cafeteria (1st floor),T07,514.0,10.0,NaN,Red,2024-2025,P0356,Mohamed Philippe,NaN,Defense,0,0,2,2,Blue
2,G023577,2025-01-13,Gym Hall,T26,1032.0,2.0,NaN,Blue,2024-2025,P0267,Luc Rossi,21.0,Attack,2,0,1,1,Red
3,G020644,2025-11-11,Salle Polyvalente,T21,310.0,6.0,NaN,Blue,2024-2025,P0208,Morgan Rossi,21.0,Defense,0,0,5,1,Blue
4,G011677,2023-09-30,Campus - Cafeteria,T26,360.0,3.0,NaN,Blue,2025-2026,P0027,Casey Kim,26.0,Attack,0,0,5,8,Red
5,G019911,2023-07-04,Gym Hall,T26,300.0,9.0,NaN,Red,2023-2024,P0181,Chris Blanc,21.0,Defense,7,0,3,5,Red
6,G015349,2023-11-16,Ynov Toulouse,T24,1080.0,0.0,NaN,Blue,2024-2025,P0734,Antoine Garcia,NaN,Defense,0,0,2,7,Red
7,G004849,2025-11-16,Lab 204,T20,491.0,7.0,NaN,None,2024-2025,P0349,Paul Clement,NaN,Defense,4,0,0,2,Red
8,G022917,2023-03-20,Bar Le Foos,T24,877.0,8.0,NaN,Red,2025-2026,P0168,Pierre Durand,NaN,Defense,0,0,4,0,Blue
9,G000463,2024-02-04,Student House,T18,966.0,0.0,NaN,Blue,2023-2024,P0022,Morgan Rousseau,34.0,Defense,1,0,5,2,Blue


In [38]:
# Informations sur le dataset
print("ℹ️  Informations sur le dataset nettoyé:\n")
df_final.info()

ℹ️  Informations sur le dataset nettoyé:

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0 to 100199
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   game_id                100000 non-null  object        
 1   game_date              99999 non-null   datetime64[ns]
 2   location               100000 non-null  object        
 3   table_id               100000 non-null  object        
 4   game_duration_seconds  99999 non-null   float64       
 5   final_score_red        99999 non-null   float64       
 6   final_score_blue       15600 non-null   float64       
 7   winner                 95338 non-null   object        
 8   season                 100000 non-null  object        
 9   player_id              100000 non-null  object        
 10  player_name            100000 non-null  object        
 11  player_age             60206 non-null   float64       
 12  player_

In [39]:
# Statistiques descriptives
print("📊 Statistiques descriptives:\n")
df_final.describe()

📊 Statistiques descriptives:



,game_date,game_duration_seconds,final_score_red,final_score_blue,player_age,player_goals,player_own_goals,player_assists,player_saves
count,99999,99999.000000,99999.00000,15600.000000,60206.000000,100000.000000,100000.000000,100000.000000,100000.00000
mean,2024-06-30 09:42:37.789577984,1049.983360,5.00546,4.977500,23.840647,2.453110,0.029380,2.503660,4.99737
min,2023-01-01 00:00:00,300.000000,0.00000,0.000000,16.000000,0.000000,0.000000,0.000000,0.00000
25%,2023-10-02 00:00:00,574.000000,2.00000,2.000000,20.000000,0.000000,0.000000,1.000000,2.00000
50%,2024-07-01 00:00:00,849.000000,5.00000,5.000000,21.000000,2.000000,0.000000,3.000000,5.00000
75%,2025-03-30 00:00:00,1140.000000,8.00000,8.000000,23.000000,4.000000,0.000000,4.000000,8.00000
max,2025-12-31 00:00:00,3599.000000,10.00000,10.000000,45.000000,10.000000,2.000000,5.000000,10.00000
std,NaN,739.705252,3.15610,3.176064,6.873113,2.501224,0.218443,1.710092,3.16212


## 💾 Sauvegarde du Dataset Nettoyé

In [40]:
# Sauvegarder le dataset nettoyé
output_path = 'babyfoot_dataset_cleaned.csv'
df_final.to_csv(output_path, index=False, encoding='utf-8')

print(f"💾 Dataset nettoyé sauvegardé dans: {output_path}")
print(f"   Taille du fichier: {len(df_final)} lignes x {len(df_final.columns)} colonnes")

💾 Dataset nettoyé sauvegardé dans: babyfoot_dataset_cleaned.csv
   Taille du fichier: 100000 lignes x 18 colonnes


## 📈 Rapport de Nettoyage Final

In [41]:
# Calculer le taux de rétention
import sys
sys.path.insert(0, '../../ressources')
df_original = pd.read_csv('../../ressources/babyfoot_dataset.csv')
original_count = len(df_original)
cleaned_count = len(df_final)
retention_rate = (cleaned_count / original_count * 100)

print("="*60)
print("📋 RAPPORT DE NETTOYAGE FINAL")
print("="*60)
print(f"Lignes originales:              {original_count:,}")
print(f"Doublons supprimés:             {duplicates_removed:,}")
print(f"Lignes invalides exclues:       {invalid_count:,}")
print(f"Lignes finales:                 {cleaned_count:,}")
print(f"Taux de rétention:              {retention_rate:.2f}%")
print("="*60)
print()
print("✅ TRANSFORMATIONS APPLIQUÉES:")
print(f"   ✓ Dates standardisées")
print(f"   ✓ Scores extraits et nettoyés")
print(f"   ✓ Gagnants standardisés (Red/Blue/Draw)")
print(f"   ✓ Noms de joueurs corrigés")
print(f"   ✓ Âges convertis en nombres")
print(f"   ✓ Rôles standardisés (Attack/Defense)")
print(f"   ✓ Couleurs d'équipe normalisées")
print(f"   ✓ Durées converties en secondes")
print(f"   ✓ Saisons standardisées (YYYY-YYYY)")
print(f"   ✓ Doublons supprimés")
print(f"   ✓ Valeurs manquantes traitées")
print("="*60)
print()
print("🎯 STATISTIQUES CLÉS:")
print(f"   Parties uniques:        {df_final['game_id'].nunique():,}")
print(f"   Joueurs uniques:        {df_final['player_id'].nunique():,}")
print(f"   Saisons uniques:        {df_final['season'].nunique()}")
print(f"   Score moyen Rouge:      {df_final['final_score_red'].mean():.2f}")
print(f"   Score moyen Bleu:       {df_final['final_score_blue'].mean():.2f}")
print(f"   Durée moyenne:          {df_final['game_duration_seconds'].mean()/60:.1f} min")
print("="*60)
print()
print("📅 SAISONS DANS LE DATASET:")
for season, count in df_final['season'].value_counts().items():
    print(f"   {season}: {count:,} lignes ({count/len(df_final)*100:.1f}%)")
print("="*60)
print()
print("✅ NETTOYAGE TERMINÉ AVEC SUCCÈS!")
print(f"   Dataset prêt pour l'analyse: {output_path}")

📋 RAPPORT DE NETTOYAGE FINAL
Lignes originales:              100,200
Doublons supprimés:             200
Lignes invalides exclues:       0
Lignes finales:                 100,000
Taux de rétention:              99.80%

✅ TRANSFORMATIONS APPLIQUÉES:
   ✓ Dates standardisées
   ✓ Scores extraits et nettoyés
   ✓ Gagnants standardisés (Red/Blue/Draw)
   ✓ Noms de joueurs corrigés
   ✓ Âges convertis en nombres
   ✓ Rôles standardisés (Attack/Defense)
   ✓ Couleurs d'équipe normalisées
   ✓ Durées converties en secondes
   ✓ Saisons standardisées (YYYY-YYYY)
   ✓ Doublons supprimés
   ✓ Valeurs manquantes traitées

🎯 STATISTIQUES CLÉS:
   Parties uniques:        25,002
   Joueurs uniques:        803
   Saisons uniques:        3
   Score moyen Rouge:      5.01
   Score moyen Bleu:       4.98
   Durée moyenne:          17.5 min

📅 SAISONS DANS LE DATASET:
   2024-2025: 59,820 lignes (59.8%)
   2025-2026: 20,296 lignes (20.3%)
   2023-2024: 19,884 lignes (19.9%)

✅ NETTOYAGE TERMINÉ AVEC SUCC

## 🎉 Prochaines Étapes

Le dataset est maintenant nettoyé et prêt pour l'analyse !

**Prochaines étapes recommandées:**
1. Ouvrir `data_analysis.py` ou `exploration.ipynb` pour l'analyse exploratoire
2. Répondre aux 3 questions du défi Data Science:
   - Top 10 des buteurs
   - Top 5 des défenseurs
   - Influence du choix du camp
3. Créer des visualisations
4. Partager les résultats avec les autres équipes

**Fichier généré:**
- `babyfoot_dataset_cleaned.csv` - Dataset propre et exploitable